In [ ]:
import numpy as np
import sklearn 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from scipy.stats import norm
from scipy.optimize import minimize 
import random
import time
import timeit
import matplotlib.pyplot as plt

import pyswarms as ps
from pyswarms.discrete import binary as bi

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
#import filetxt and assign sample x and sample y

batch_num = 3
num_bits = 50
initial_path = f"D:/Thickness_optimization/Thickness_optimization_binary_thickness/Initial_data/bat{batch_num}/"
filename='training_data_thickness_binary_sample{n}_{bits}'.format(n=batch_num,bits = num_bits+6)
f_1=open(initial_path+filename+'.txt')
data_array=np.loadtxt(f_1)
sample_x=np.float32(data_array[:,1:data_array.shape[1]])
sample_y=np.float32(data_array[:,0])


In [ ]:
#Defining Optimizer

def GPRoptimizer(obj_func, initial_theta, bounds):
   
    def get_fun(theta): 
   #reshape X_star to fit the later input
        fun_v=obj_func(theta, eval_gradient=True)
   #print(str(fun_v))
        return fun_v
   
    funval = minimize(get_fun, initial_theta, method='L-BFGS-B', jac=True, bounds=bounds,options={'maxiter':30000})
    theta_opt=funval.x
   
    def get_fun_eval(theta):
   #reshape X_star to fit the later input
       fun_eval=obj_func(theta, eval_gradient=False)
       return fun_eval
    func_min=get_fun_eval(theta_opt)
    return theta_opt, func_min

kernel = 1 * RBF(length_scale=1, length_scale_bounds=(1e-2, 1e2))
#-------------------------------------------------------------------------------

gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=1,optimizer=GPRoptimizer)
gaussian_process.fit(sample_x, sample_y)
gaussian_process.kernel_

In [ ]:
# ABC test script

def acq_fun_EI_ABC(input_x):
    input_x = np.array(input_x)
    input_x = input_x.reshape(1,-1)
    y_sample_data = sample_y
    y_prediction_mean, y_prediciton_var = gaussian_process.predict(input_x, return_std=True) #x_star[binary]
    
    def EI(mean, var, ff):
        Z = (ff.min() - mean) / np.sqrt(var)
        ei = (ff.min() - mean) * norm.cdf(Z) + np.sqrt(var) * norm.pdf(Z)
        return ei
    ei = EI(y_prediction_mean, y_prediciton_var, y_sample_data)
    return -ei 


In [ ]:
from beecolpy import bin_abc
import matlab.engine
eng = matlab.engine.start_matlab()

In [ ]:
#ABC test script

def test_ABC_binary(obj_function,bits,colony,iteration):
    bits_count = bits #total bits
    bin_abc_obj = bin_abc(obj_function,bits_count,method='am',
                          colony_size=colony,
                          scouts=0.5,
                          iterations=iteration,
                          min_max='min',
                          nan_protection=True,
                          transfer_function='sigmoid',
                          best_model_iterations=0,
                          log_agents=True)
    
    start_time_ABC = time.time()
    bin_abc_obj.fit() #Execute the algorithm
    end_time_ABC = time.time()
    
    execution_time_ABC = round(end_time_ABC - start_time_ABC,3)
    
    #If you want to get the obtained solution after execute the fit() method:
    solution = bin_abc_obj.get_solution()
    iterations = bin_abc_obj.get_status()[0]
    scout = bin_abc_obj.get_status()[1]
    nan_events = bin_abc_obj.get_status()[2]
    binary_list = [int(value) for value in solution]
    #print("solution: \n",solution_x)
    #print("iteration: \n",iterations)
    #print("scout: \n",scout)
    #print("NaN events: \n",nan_events)
    return execution_time_ABC, binary_list

#test_ABC_binary(acq_fun_EI_ABC,26,2**6,10)

In [ ]:

history_FOM = [] #history 그래프를 그리기 위해, 각 iter당 min값 저장
history_x = []
iteration_BO = 2000

colony = 2**6
iteration_ABC = 1000

fom_min_junk_container = np.zeros(1)
qv_vector_size=int(sample_x.shape[1])

for i in range(1,iteration_BO+1):
    
    
    print("\n iteration: ",i)
    start_time = time.time()
    
    #Call instace of ABC
    execution_time_ABC, x_initial_guess = test_ABC_binary(acq_fun_EI_ABC,qv_vector_size,colony,iteration_ABC)
    print("execution time of ABC :",execution_time_ABC)
    print("x+1 : ",x_initial_guess)
    
    for k in range(sample_x.shape[0]):
        if(np.array_equal(x_initial_guess[:],sample_x[k,:])):
            print("random!: ",i)
            for j in range(qv_vector_size):
                qv_random = random.random()
                if (qv_random>1/2):
                    x_initial_guess[j]=1
                else:
                    x_initial_guess[j]=0

    sample_x=np.vstack((sample_x,x_initial_guess)) #sample_x에 (x+1)추가
    x_list = x_initial_guess 
    
    for ind in range(qv_vector_size):
        x_list[ind]=int(x_list[ind])

    x_mat = matlab.double(x_list)

    #-----------------------here TMM-------------------------------------------
    start_time_TMM = time.time()
    fom_min_junk_container[0]=eng.adaptive_scheme_FOM(x_mat)  #matlab으로 계산한 x+1의 fom값
    end_time_TMM = time.time()
    execution_time_TMM = round((end_time_TMM - start_time_TMM),3)

    #------------------------------------------------------------------
    
    print("for x+1, fom: ", fom_min_junk_container[0])
    sample_y=np.append(sample_y,np.array([fom_min_junk_container[0]]))

    #----------------------------here GP------------------------------
    start_time_GP = time.time()
    gaussian_process.fit(sample_x, sample_y)
    end_time_GP = time.time()
    execution_time_GP = round((end_time_GP - start_time_GP),3)

    print(f"Execution time Gaussian: {execution_time_GP} seconds")
    #------------------------------------------------------------------
    
    history_FOM=np.append(history_FOM,sample_y.min())
    min_fom_x = sample_x[sample_y.argmin(),:]
    history_x.append(min_fom_x)
    
    print("minimum fom address: ",min_fom_x)
    print("minimum fom: ",sample_y.min())
    
    
     #--------------------save_result_txt------------------------------
    result2_str = ' '.join(map(str, x_list))
    result1_str = f"{fom_min_junk_container[0]:1.15f}"
    save_fom = '{size}_layers_{iters}_iters_GPR_ABC_thickness_binary_{n}_{col}_{iterABC}'.format(size = num_bits, iters = iteration_BO,n=batch_num,col = int(np.log2(colony)),iterABC = int(iteration_ABC))
    file_path = f'D:/Thickness_optimization/Thickness_optimization_binary_thickness/BO_ABC_binary_opt_result/bat{batch_num}/'
    if (i==1):
        sample_y_reshaped = sample_y.reshape(-1, 1)
        merged_array = np.hstack((sample_y_reshaped, sample_x))
        N = sample_x.shape[1]
        format_txt=' '.join(['%1.15f'] + ['%d']*N)
        np.savetxt(file_path+save_fom+'.txt', merged_array, fmt = format_txt, delimiter=' ')
        
    else:
        with open(file_path+save_fom+'.txt','a') as writing_txt:
            writing_txt.write(f"{result1_str} {result2_str}\n")
        
    end_time = time.time()
    time_record = int(end_time - start_time)
    print("total time: ",time_record,'sec')
    save_time = '{size}_layers_{iters}_iters_time_record_GPR_ABC_thickness_binary_{n}_{col}_{iterABC}'.format(size = num_bits, iters = iteration_BO,n=batch_num,col = int(np.log2(colony)),iterABC = int(iteration_ABC))
    
    with open(file_path+save_time+'.txt','a') as time_txt:
        time_txt.write(f"{i} {execution_time_GP} {execution_time_ABC} {execution_time_TMM} {time_record}\n")
    